# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Dfapp").getOrCreate()

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [3]:
df=spark.read.option("header","true").csv("F:\\datasets\\googleplaystore.csv",header=True,inferSchema=True)
df.show(5)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|             1.2.4|4.0.3 and up|
|Ske

As well as the schema to make sure all the column types were correctly infered

In [4]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [5]:
from pyspark.sql.types import IntegerType, FloatType
newdf = df.withColumn("Rating", df["Rating"].cast(FloatType())) \
            .withColumn("Reviews", df["Reviews"].cast(IntegerType())) \
            .withColumn("Price", df["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [6]:
df.createOrReplaceTempView("tempview")
newdf.createOrReplaceTempView('tempview2')

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [7]:
g1=spark.sql('select App from tempview2 where rating >4.1')
g1.show()

+--------------------+
|                 App|
+--------------------+
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|        ibis Paint X|
|Superheroes Wallp...|
|Mcqueen Coloring ...|
|HD Mickey Minnie ...|
|Harley Quinn wall...|
|Colorfit - Drawin...|
+--------------------+
only showing top 20 rows



## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [8]:

i=spark.sql('select App,Rating from tempview2 where Rating>4.5 and Category="COMICS"')
i.show()

+---------------------+------+
|                  App|Rating|
+---------------------+------+
| Manga Master - Be...|   4.6|
| GANMA! - All orig...|   4.7|
| Röhrich Werner So...|   4.7|
| Unicorn Pokez - C...|   4.8|
| Manga - read Thai...|   4.6|
| Dragon Ball Wallp...|   4.7|
| Children's cartoo...|   4.6|
|【Ranobbe complete...|   NaN|
|       Faustop Sounds|   4.7|
| Hojiboy Tojiboyev...|   5.0|
| Best Wallpapers B...|   4.7|
| Laftel - Watching...|   4.6|
|            WebComics|   4.8|
| Superheroes, Marv...|   5.0|
|       Pepsi Cards DC|   NaN|
+---------------------+------+



## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [9]:
o=spark.sql('select category,sum(reviews)from tempview2 group by category')
o.show(1)



+--------+------------+
|category|sum(reviews)|
+--------+------------+
|  EVENTS|      161018|
+--------+------------+
only showing top 1 row



## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [10]:
j=spark.sql('select App,Reviews from tempview2 order by Reviews desc ')
j.show(1)

+--------+--------+
|     App| Reviews|
+--------+--------+
|Facebook|78158306|
+--------+--------+
only showing top 1 row



## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [102]:
l=spark.sql('select App from tempview2 where app like "%dating%"').show()

+--------------------+
|                 App|
+--------------------+
|Meet, chat & date...|
|Friend Find: free...|
|Spine- The dating...|
|Princess Closet :...|
|happn – Local dat...|
+--------------------+



## 6. Use SQL Transformer to display how many free apps there are in this list

In [12]:

from pyspark.ml.feature import SQLTransformer
res=SQLTransformer(statement="select app from __THIS__ where Type='Free'")
res.transform(newdf).show()

+--------------------+
|                 app|
+--------------------+
|Photo Editor & Ca...|
| Coloring book moana|
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Pixel Draw - Numb...|
|Paper flowers ins...|
|Smoke Effect Phot...|
|    Infinite Painter|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Tattoo Name On My...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Learn To Draw Kaw...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|FlipaClip - Carto...|
|        ibis Paint X|
+--------------------+
only showing top 20 rows



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [14]:
pd=spark.sql('select App, Genres from tempview2 order by genres desc')
pd.show(1)

+-----------+------+
|        App|Genres|
+-----------+------+
|Word Search|  Word|
+-----------+------+
only showing top 1 row



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [15]:
p=spark.sql('select app,Genres from tempview2 where Reviews>100 and Genres="Tools"')
p.show()

+--------------------+------+
|                 app|Genres|
+--------------------+------+
|   Moto File Manager| Tools|
|              Google| Tools|
|    Google Translate| Tools|
|        Moto Display| Tools|
|      Motorola Alert| Tools|
|     Motorola Assist| Tools|
|Cache Cleaner-DU ...| Tools|
|  Moto Suggestions ™| Tools|
|          Moto Voice| Tools|
|          Calculator| Tools|
|         Device Help| Tools|
|     Account Manager| Tools|
|             myMetro| Tools|
|        File Manager| Tools|
|           My Telcel| Tools|
|Calculator - free...| Tools|
| ASUS Sound Recorder| Tools|
|  iWnn IME for Nexus| Tools|
|Samsung Max - Dat...| Tools|
|          ZenUI Help| Tools|
+--------------------+------+
only showing top 20 rows



## That's all folks! Great job!